# Predicting Video Game Sales 


## Overview
1. Understanding the shape and distribution of the data 
2. Exploratory Data Analysis
3. Feature Engineering
4. Data Pre-processing
5. Model Building and Tuning
6. Final Model

In [ ]:
# Load Libaries Needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
# Read the data
data_path='/kaggle/input/videogamesales/vgsales.csv'
df=pd.read_csv(data_path)
# Print the first 5 rows
df.head()

In [ ]:
# Look at all the variables in the columns
df.columns

In [ ]:
#Explore the data type of each column
df.info()

In [ ]:
# Get a snapshot of the data's central tendencies 
df.describe()

In [ ]:
# Let's see how the Global Sales are distritubed for all the data 
f,ax=plt.subplots(figsize=(10,5))
sns.distplot(df['Global_Sales'])
plt.title("Distribution of Global Sales for Video Games")


In [ ]:
# Calculate Skewness and Kurtosis 
print ("Skewness: " + str(df['Global_Sales'].skew()))
print ("Kurtosis: "+str(df['Global_Sales'].kurtosis()))

We can see from the plot above that the Global Sales for video games is significantly right skewed with a heavy tail. This may indicate that most video games are quite relatively unsuccessful, however there are few that really break out and have a ton of success. 

In addition to this, the large value for Kurtosis indicates the presence of outliers. This is something we will need to consider later when making our model.

# **Exploring the breakdown of all the numerical and categorical variables**


## Numerical Variables:

* We will create distribution plots to see how each of these are spread out in our dataset.
* We will analyse the correlation between these variables 

## Categorical Variables:
* We will create bar charts to see the distrbution of the data broken down by different variables 

In [ ]:
#Numerical Variables 
num= ['Year','NA_Sales','EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
df_num=df[num]
for var in df_num:
    if var!='Global_Sales':
        f,ax=plt.subplots(figsize=(5,5))
        sns.distplot(df_num[var])


In [ ]:
#Calculate Skewness and Kurtosis for each numerical variable
for var in df_num: 
    if var != "Global_Sales":
        print ('Skewness for '+ var + ' is: '+ str(df[var].skew()))
        print ('Kurtosis for '+ var + ' is: '+ str(df[var].kurtosis()))

In [ ]:
#Calculate Correlation between numeric variables
sns.heatmap(df_num.corr(),annot=True,cmap="YlGnBu")

In [ ]:
#Categorical Variables
cat=['Platform','Genre', 'Publisher']
df_cat=df[cat]
for var in df_cat:
    f,ax=plt.subplots(figsize=(16,8))
    sns.barplot(df_cat[var].value_counts().index,df_cat[var].value_counts()).set_title('Distrubution of ' + var)
    plt.xlabel(var)
    plt.ylabel('Frequency')
    plt.show()

Some of the categorical variables are quite messy. Therefore, we may want to do some feature engineering to get a better representation of this in our model.

# Exploratory Data Analysis

We shall explore some key themes in this dataset:

1. Sales across different platforms 
2. Sales over the years
3. Genres that are the best selling
4. Most successful publishers
5. Comparing the sales data for games in the total series (FIFA, Call of Duty, GTA, etc.)
6. Console vs. Portable Device Video Game Sales

In [ ]:
# Sales across different platforms:
f,ax=plt.subplots(figsize=(18,6))
sns.boxplot(df['Platform'],df['Global_Sales'])
plt.show()

#Calculate average sale price for each platform:
df2=df.groupby(['Platform'])['Platform','Global_Sales'].mean().sort_values(by='Global_Sales', ascending=False).reset_index()
df2

In [ ]:
# Sales over the years 
df3 =df.groupby(['Year'])['Global_Sales'].mean().sort_values(ascending=False).reset_index()
print(df3)

# Scatter Plot
f,ax=plt.subplots(figsize=(18,6))
plt.scatter(df['Year'],df['Global_Sales'])


In [ ]:
# Best Selling Genres:
df4 =df.groupby(['Genre'])['Genre','Global_Sales'].mean().sort_values(by='Global_Sales', ascending=False).reset_index()
print(df4)

# Boxplot comparison between genres
f,ax=plt.subplots(figsize=(18,6))
sns.boxplot(df['Genre'],df['Global_Sales'])
plt.ylim([0,10])

In [ ]:
# Most Successful Publishers:
df5 =df.groupby(['Publisher'])['Publisher','Global_Sales'].mean().sort_values(by='Global_Sales', ascending=False).reset_index()
print(df5.head(10))

# Boxplot comparing Publishers:
f,ax=plt.subplots(figsize=(18,6))
sns.boxplot(df['Publisher'],df['Global_Sales'])

In [ ]:
# Comparing Sales of Major Game Series
series=['Fifa','Mario','Call of Duty','Grand Theft Auto', 'Pokemon','Halo','Wii','NBA']

for ser in series:
    M = df[df['Name'].str.contains(ser, regex=False, case=False, na=False)].copy()
    f,ax=plt.subplots(figsize=(16,5))
    plt.scatter(M['Year'],M['Global_Sales'])
    plt.title(ser+' Sales Over Time')
    plt.xlabel('Year')
    plt.ylabel('Global Sales')
    plt.show()



In [ ]:
# Compare average sales for each major game series
series=['Fifa','Mario','Call of Duty','Grand Theft Auto', 'Pokemon','Halo','Wii','NBA']
for ser in series:
    game_series={}
    M = df[df['Name'].str.contains(ser, regex=False, case=False, na=False)].copy()
    average= round(M['Global_Sales'].median(),2)
    correlation=M['Global_Sales'].corr(M['Year'])
    rounded_corr=round(correlation,2)
    print('The Median For {} Series is {}, its Correlation with Years is {}'.format(ser,average,rounded_corr))

In [ ]:
# Compare Type of Device Game Sales 
print(df['Platform'].unique())
# 3 types: Portable, PC and Console
PC=['PC']
Portable = ['GB','DS','GBA','3DS','PSP','PSV','WS','GG']
df_portable=df[(df.Platform =='GB')|(df.Platform =='DS')|(df.Platform =='GBA')|(df.Platform =='3DS')|(df.Platform =='PSP')|(df.Platform =='PSV')
              |(df.Platform =='WS')|(df.Platform =='GG')]
df_PC=df[(df.Platform=='PC')]

df_console= df[(df.Platform =='Wii')|(df.Platform =='NES')|(df.Platform =='X360')|(df.Platform =='PS3')|(df.Platform =='PS2')|(df.Platform =='SNES')
              |(df.Platform =='PS4')|(df.Platform =='N64')|(df.Platform =='PS')|(df.Platform =='XB')|(df.Platform =='2600')|(df.Platform =='XOne')
             |(df.Platform =='GC')|(df.Platform =='WiiU')|(df.Platform =='GEN')|(df.Platform =='DC')
            |(df.Platform =='SAT')|(df.Platform =='SCD')|(df.Platform =='NG')|(df.Platform =='TG16')|(df.Platform =='3DO')
            |(df.Platform =='PCFX')]
d={'Portable':df_portable,'PC': df_PC,'Console':df_console}
for name,n in d.items():
    f,ax=plt.subplots(figsize=(10,6))
    plt.scatter(n['Year'],n['Global_Sales'])
    plt.title(name + ' Device Sales Over Time')
    plt.xlabel('Year')
    plt.ylabel('Global Sales')
    plt.show()
    average= n['Global_Sales'].median()
    print ("The Median Sales for {} devices is {}".format(name,average))




# **Feature Engineering**

1. Whether it is released on a Console, Portable device or PC
2. The game series it is apart of (We consider the ones we performed EDA on)
3. Percentage of Total Sales Per Region


In [ ]:
len(df['Platform'])

In [ ]:
#Console, Portable or PC
Device=[]
for i in range(0,16598):
    if df['Platform'][i] in Portable:
        Device.append('Portable')
    elif df['Platform'][i] in PC:
        Device.append('PC')
    else:
        Device.append('Console')
df['Device']=Device
df.head()
        

In [ ]:
# Adding Feature of Game Series
series=['FIFA','Mario','Call of Duty','Grand Theft Auto', 'Pokemon','Halo','Wii','NBA']
for ser in series:
    df[ser]=df['Name'].str.contains(ser, regex=False, case=False, na=False)
df.head(10)

In [ ]:
# Percentage of Total Sales Per Region
regions_sales=['NA_Sales','EU_Sales','JP_Sales','Other_Sales']
for reg in regions_sales:
    A=df[reg]
    B=df['Global_Sales']
    df[reg+' As a percentage of Total']=round(100*(A/B),2)
df

# Data Pre-processing for Model 

Now that we have engineered 3 features that better represent our variables better. Let's start pre-processing the data so that we can apply our model to it:

**1. Splitting data into input and output variables**:

We will ignore the Video Game's Name and Platform from our dataset as we have feature engineered these variables into more useful ones.

We will also ignore the gross number of region sales and instead look at each as a percentage of the total. This is because the gross region sales are highly correlated with the global sales since they actually make it up. Therefore, if left this in our model since it would be a data leakage for our target variable. Despite this, it would be interesting to see if the distrbution of sales by region impacts the actual Global Sales number. Thus, we will look at this through expressing it as a percentage of global sales.   


**2. Dealing with missing values and cateogrical variables**

**3. Winsorization for outliers**

In our data set we can see that there is a significant outliers for global sales. We can see this in the distrbution plot of global sales where the mean is significantly lower than the maximum. The next closest global sales value is more than half of the maximum. In order to combat the impact that outliers will have on our model we will use winsorization to reduce its effect.    



In [ ]:
# Splitting Data into input and output variables
features= ['Year', 'Genre', 'Publisher','Device',
       'FIFA', 'Mario', 'Call of Duty', 'Grand Theft Auto', 'Pokemon', 'Halo',
       'Wii', 'NBA', 'NA_Sales As a percentage of Total',
       'EU_Sales As a percentage of Total',
       'JP_Sales As a percentage of Total',
       'Other_Sales As a percentage of Total']
X=df[features]
X
y=df['Global_Sales']

In [ ]:
# Checking for Missing values
print(X.isnull().sum())

In [ ]:
# Investigating which rows have missing values
df[df.isnull().any(axis=1)]
# For those with Publishers as NaN we will fill them with unknown
X.Publisher.fillna('Unknown',inplace=True)
X.Publisher.isnull().sum()

In [ ]:
# Dealing with missing year value:
df[df.Year.isnull()]
# From look at this dataset there does not seem to be a pattern for missing years. We will therefore fill it by its median since 
# this will be a better measure of center than mean due to the left skewed distrbution of Years in this dataset
f=df.Year.median()
X.Year.fillna(f,inplace=True)
# We should indicate in which records we have imputed years column with the median
X['Year was Missing']=df.Year.isnull()
X[X['Year was Missing']==True]
# Confirm all missing values are filled
X.isnull().sum()

In [ ]:
# Now let's move on to the categorical variables
# For Series Type we have already encoded it using OneHotEncoding. The only other two variables left are Genre, Publisher and Device Type
# Check the cardinality for each of these variables
for col in ['Genre','Publisher','Device']:
    print('The cardinality of {} is {}'.format(col,X[col].nunique()))

In [ ]:
# For Device Type we can use One-Hot Encoding 
from sklearn.preprocessing import OneHotEncoder
OH_Encoder=OneHotEncoder(sparse=False,handle_unknown='ignore')
low_card_col=['Device']
OH_cols_X=pd.DataFrame(OH_Encoder.fit_transform(X[low_card_col]))
OH_cols_X.index=X.index
other=X.drop(low_card_col,axis=1)
X_processed=pd.concat([other,OH_cols_X], axis=1)

In [ ]:
# For the high cardinality columns we shall use label encoding to minimise the data transformations needed to take place
high_cardinality =['Genre','Publisher']
label_X=X_processed.copy()
from sklearn.preprocessing import LabelEncoder
label_encoder= LabelEncoder()
for col in high_cardinality:
    label_X[col]=label_encoder.fit_transform(X[col])

In [ ]:
# Winsorization
from scipy.stats.mstats import winsorize
y_w=winsorize(y,limits=0.0001)
y_w

# Let's Finally Start Model Building!

Using cross validation scores we will start to evaluate different models performance on our dataset

## Baseline Model Results

1. Linear Regression: 0.670
2. Decision Tree Regressor: 0.569
3. Random Forest Regressor: 0.570
4. Bagging Regressor: 0.572
5. Extra Trees Regressor: 0.569
6. Gradient Boosting: 0.566
7. **Voting Regressor #1: 0.561**  - where we will combine the best baseline models and make them vote on the final predicted value. 



In [ ]:
# Linear Regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
lin=LinearRegression()
cv=-1*cross_val_score(lin,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
# Decision Tree Regressor
from sklearn import tree
dt=tree.DecisionTreeRegressor(max_leaf_nodes=950,random_state=1)
cv=-1*cross_val_score(dt,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
# Random Forest Regressor
from sklearn import ensemble
rf=ensemble.RandomForestRegressor(n_estimators=100,random_state=1)
cv=-1*cross_val_score(rf,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
# Bagging  Regressor
bag=ensemble.BaggingRegressor(random_state=1)
cv=-1*cross_val_score(bag,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
# Extra Trees Regressor
etr=ensemble.ExtraTreesRegressor(random_state=1,n_estimators=100)
cv=-1*cross_val_score(etr,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
# Gradient Boosting Regressor
gb=ensemble.GradientBoostingRegressor(random_state=1)
cv=-1*cross_val_score(gb,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

In [ ]:
#Now let's try combining all the models with a average MAE of lower than 0.57 in a Voting Regressor:
from sklearn.ensemble import VotingRegressor
voting_reg=VotingRegressor(estimators=[('dt',dt),('rf',rf),('gb',gb),('bag',bag),('etr',etr)])
cv=-1*cross_val_score(voting_reg,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print(cv)
print(cv.mean())

# Model Tuning

We will tune the following model to see if we want improve its performance:
* Random Forest

In [ ]:
# Model Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

def model_performance(model,name):
    print(name)
    print('Best Score: {}'.format(model.best_score_))
    print('Best Parameters: {}'.format(model.best_params_))

In [ ]:
# Random Forest Random Search
rf
param={
    'n_estimators':[100,500,1000]}
reg_rf_rs=GridSearchCV(rf, param_grid=param, cv = 5, verbose = True, n_jobs = -1)
best_reg_rf_rnd = reg_rf_rs.fit(label_X,y_w)
model_performance(best_reg_rf_rnd,'Random Forest')

In [ ]:
# Most Important Features for Random Forest
best_rf = best_reg_rf_rnd.best_estimator_.fit(label_X,y_w)
feat_importances = pd.Series(best_rf.feature_importances_, index=label_X.columns)
feat_importances.plot(kind='barh')

In [ ]:
# Best RF Model
best_rf=best_reg_rf_rnd.best_estimator_

#Developing Voting Model
voting_reg_1 = VotingRegressor(estimators=[('dt',dt),('rf',best_rf),('gb',gb),('bag',bag),('etr',etr)])
cv=-1*cross_val_score(voting_reg_1,label_X,y_w,cv=5,scoring='neg_mean_absolute_error')
print('Cross Val Score: {}'.format(cv))
print('Average Cross Val Score: {}'.format(cv.mean()))

In [ ]:
# Try and improve weights for the best voting model:
params = {'weights' : [[1,1,1,1,1],[1,2,1,1,1],[1,1,2,1,1],[1,2,2,1,1]]}

vote_weight = GridSearchCV(voting_reg_1, param_grid = params, cv = 5, verbose = True, n_jobs = -1)
best_reg_weight = vote_weight.fit(label_X,y_w)
model_performance(best_reg_weight,'VR Weights')

best_vote=best_reg_weight.best_estimator_

# Final Model Building

We will now build our final model using the tuned **voting regressor model** and test it on 1000 rows of the original dataset as our final test data in order to get key model metrics such as MAE and R2.

## Summary of Final Model Buliding Results
* MAE:
* R2: 



In [ ]:
# Split Data into Train and Test
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(label_X,y_w,random_state=1,test_size=1000)

# Voting Regressor
# Train Model and Make predictions
best_vote.fit(train_X,train_y)
pred=best_vote.predict(test_X)

# Compute MAE and R2
from sklearn import metrics
mae=metrics.mean_absolute_error(test_y,pred)
r2=metrics.r2_score(test_y,pred)
print('The Mean Absolute Error for our final model is {}'.format(mae))
print('The R-squared score for our final model is {}'.format(r2))

In [ ]:
feature_name=(test_X.columns.tolist())
feature_name[-3]='Console'
feature_name[-2]='PC'
feature_name[-1]='Portable'

In [ ]:
# Explore feature importance
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(best_vote,random_state=1).fit(test_X, test_y)
eli5.show_weights(perm,feature_names = feature_name)

We can see in the table above that the top 3 features/predictors of a video game's global sales is:
1. Percentage of Total Sales in Japan
2. Percentage of Total Sales in North America
3. Mario Games

This provides really interesting insights into where video games should be targeted if a company wants to maxmise its sales. 
I will discussed these insights and my interpretations further in [my blog post](https://mirpurishail.wixsite.com/portfolio/data-blogs). 


# References

[The Dataset](https://www.kaggle.com/gregorut/videogamesales)

[Winsorization](https://www.youtube.com/watch?v=WLFQxoXn_uw) 

[Model Building and Tuning](https://www.kaggle.com/kenjee/titanic-project-example)

[Permutation Importance](https://www.kaggle.com/nitindatta/fifa-in-depth-analysis-with-linear-regression)

[SK Learn Documentation](https://scikit-learn.org/stable/index.html)